In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("sparkstreamWordCount")


from pyspark.sql import SparkSession
spark =SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc= spark.sparkContext 

22/03/11 23:08:15 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/11 23:08:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/11 23:08:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/11 23:08:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#structured streaming

linestreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

22/03/11 23:08:29 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
linestreamDf.printSchema()

#linestreamDf.show()

root
 |-- value: string (nullable = true)



In [6]:
from pyspark.sql.functions import split, explode, col

wordsDf = linestreamDf.select(explode(split(linestreamDf.value, " ")).alias("word"))
wordsDf.printSchema()

root
 |-- word: string (nullable = true)



In [7]:
#word count by group
wordCountDf = wordsDf.groupBy("word").count()

In [9]:
echoOnconsole = wordCountDf.writeStream.outputMode("complete").format("console").start()

22/03/11 23:27:04 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4d9b558b-3db8-438c-ad4b-1ecc63e08fbe. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|welcome|    1|
|  hello|    1|
+-------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|welcome|    1|
|  kafka|    1|
|  hello|    1|
+-------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|welcome|    1|
|  kafka|    1|
|  hello|    2|
| python|    1|
+-------+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|welcome|    1|
|  kafka|    2|
|  hello|    3|
| python|    1|
+-------+-----+



In [10]:
echoOnconsole = wordCountDf.writeStream.outputMode("update").format("console").start()

22/03/11 23:31:04 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f1eadfca-d0b8-4b30-b426-fdda5154faca. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+

